# Generators - where to find them
`Generator` - what do you associate this word with? Perhaps it refers to some electronic device. Or perhaps it refers to a heavy and serious machine designed to produce power, electrical or other.

A Python generator is `a piece of specialized code able to produce a series of values, and to control the iteration process`. This is why generators are very often called `iterators`, and although some may find a very subtle distinction between these two, we'll treat them as one.

You may not realize it, but you've encountered generators many, many times before. Take a look at the very simple snippet:

In [ ]:
for i in range(5):
    print(i)

The `range()` function is, in fact, a generator, which is (in fact, again) an iterator.

What is the difference?

A function returns one, well-defined value - it may be the result of a more or less complex evaluation of, e.g., a polynomial, and is invoked once - only once.

A generator `returns a series of values`, and in general, is (implicitly) invoked more than once.

In the example, the `range()` generator is invoked six times, providing five subsequent values from zero to four, and finally signaling that the series is complete.

The above process is completely transparent. Let's shed some light on it. Let's show you the `iterator protocol`.

# Generators - where to find them: continued
The `iterator protocol is a way in which an object should behave to conform to the rules imposed by the context of the for and in statements`. An object conforming to the iterator protocol is called an `iterator`.

An iterator must provide two methods:

  - `__iter__()` which should `return the object itself` and which is invoked once (it's needed for Python to successfully start the iteration)
  - `__next__()` which is intended to `return the next value` (first, second, and so on) of the desired series - it will be invoked by the `for`/`in` statements in order to pass through the next iteration; if there are no more values to provide, the method should `raise the StopIteration exception`.
Does it sound strange? Not at all. Look at the example in the editor.

We've built a class able to iterate through the first `n` values (where `n` is a constructor parameter) of the Fibonacci numbers.

Let us remind you - the Fibonacci numbers ($Fib_{i}$) are defined as follows:

$Fib_{1}$ = 1

$Fib_{2}$ = 1

$Fib_{i}$ = $Fib_{i-1}$ + $Fib_{i-2}$

In other words:

  - the first two Fibonacci numbers are equal to 1;
  - any other Fibonacci number is the sum of the two previous ones (e.g., $Fib_{3}$ = 2, $Fib_{4}$ = 3, $Fib_{5}$ = 5, and so on)
  


In [2]:
class Fib:
    def __init__(self, nn):
        print("__init__")
        self.__n = nn
        self.__i = 0
        self.__p1 = self.__p2 = 1

    def __iter__(self):
        print("__iter__")
        return self

    def __next__(self):
        print("__next__")				
        self.__i += 1
        if self.__i > self.__n:
            raise StopIteration
        if self.__i in [1, 2]:
            return 1
        ret = self.__p1 + self.__p2
        self.__p1, self.__p2 = self.__p2, ret
        return ret


for i in Fib(10):
    print(i)

__init__
__iter__
__next__
1
__next__
1
__next__
2
__next__
3
__next__
5
__next__
8
__next__
13
__next__
21
__next__
34
__next__
55
__next__


Let's dive into the code:

  - lines 2 through 6: the class constructor prints a message (we'll use this to trace the class's behavior), prepares some variables (`__n` to store the series limit, `__i` to track the current Fibonacci number to provide, and `__p1` along with `__p2` to save the two previous numbers);

  - lines 8 through 10: the `__iter__` method is obliged to return the iterator object itself; its purpose may be a bit ambiguous here, but there's no mystery; try to imagine an object which is not an iterator (e.g., it's a collection of some entities), but one of its components is an iterator able to scan the collection; the `__iter__` method should `extract the iterator and entrust it with the execution of the iteration protocol`; as you can see, the method starts its action by printing a message;

  - lines 12 through 21: the `__next__` method is responsible for creating the sequence; it's somewhat wordy, but this should make it more readable; first, it prints a message, then it updates the number of desired values, and if it reaches the end of the sequence, the method breaks the iteration by raising the StopIteration exception; the rest of the code is simple, and it precisely reflects the definition we showed you earlier;

  - lines 24 and 25 make use of the iterator.

The code produces the following output:
```s
__init__
__iter__
__next__
1
__next__
1
__next__
2
__next__
3
__next__
5
__next__
8
__next__
13
__next__
21
__next__
34
__next__
55
__next__
```

Look:

  - the iterator object is instantiated first;
  - next, Python invokes the `__iter__` method to get access to the actual iterator;
  - the `__next__` method is invoked eleven times - the first ten times produce useful values, while the eleventh terminates the iteration.


# Generators - where to find them: continued
The previous example shows you a solution where the `iterator object is a part of a more complex class`.

The code isn't really sophisticated, but it presents the concept in a clear way.

Take a look at the code in the editor.

In [1]:
class Fib:
    def __init__(self, nn):
        self.__n = nn
        self.__i = 0
        self.__p1 = self.__p2 = 1

    def __iter__(self):
        print("Fib iter")
        return self

    def __next__(self):
        self.__i += 1
        if self.__i > self.__n:
            raise StopIteration
        if self.__i in [1, 2]:
            return 1
        ret = self.__p1 + self.__p2
        self.__p1, self.__p2 = self.__p2, ret
        return ret

class Class:
    def __init__(self, n):
        self.__iter = Fib(n)

    def __iter__(self):
        print("Class iter")
        return self.__iter;


object = Class(8)

for i in object:
    print(i)

Class iter
1
1
2
3
5
8
13
21


We've built the `Fib` iterator into another class (we can say that we've composed it into the `Class` class). It's instantiated along with `Class`'s object.

The object of the class may be used as an iterator when (and only when) it positively answers to the `__iter__` invocation - this class can do it, and if it's invoked in this way, it provides an object able to obey the iteration protocol.

This is why the output of the code is the same as previously, although the object of the `Fib` class isn't used explicitly inside the `for` loop's context.

# The yield statement
The iterator protocol isn't particularly difficult to understand and use, but it is also indisputable that the `protocol is rather inconvenient`.

The main discomfort it brings is `the need to save the state of the iteration between subsequent __iter__ invocations`.

For example, the `Fib` iterator is forced to precisely store the place in which the last invocation has been stopped (i.e., the evaluated number and the values of the two previous elements). This makes the code larger and less comprehensible.

This is why Python offers a much more effective, convenient, and elegant way of writing iterators.

The concept is fundamentally based on a very specific and powerful mechanism provided by the yield keyword.

You may think of the `yield` keyword as a smarter sibling of the `return` statement, with one essential difference.

Take a look at this function:
```py
def fun(n):
    for i in range(n):
        return i
```

It looks strange, doesn't it? It's clear that the `for` loop has no chance to finish its first execution, as the `return` will break it irrevocably.

Moreover, invoking the function won't change anything - the `for` loop will start from scratch and will be broken immediately.

We can say that such a function is not able to save and restore its state between subsequent invocations.

This also means that a function like this `cannot be used as a generator`.

We've replaced exactly one word in the code - can you see it?
```py
def fun(n):
    for i in range(n):
        yield i
```

We've added `yield` instead of `return`. This little amendment `turns the function into a generator`, and executing the `yield` statement has some very interesting effects.

First of all, it provides the value of the expression specified after the `yield` keyword, just like `return`, but doesn't lose the state of the function.

All the variables' values are frozen, and wait for the next invocation, when the execution is resumed (not taken from scratch, like after `return`).

There is one important limitation: such a `function should not be invoked explicitly` as - in fact - it isn't a function anymore; `it's a generator object`.

The invocation will `return the object's identifier`, not the series we expect from the generator.

Due to the same reasons, the previous function (the one with the `return` statement) may only be invoked explicitly, and must not be used as a generator.

# How to build a generator
Let us show you the new generator in action.

This is how we can use it:
```py
def fun(n):
    for i in range(n):
        yield i


for v in fun(5):
    print(v)
```

Can you guess the output?

A:
```s
0
1
2
3
4
```

# How to build your own generator
What if you need a `generator to produce the first n powers of 2`?

Nothing easier. Just look at the code below:

In [3]:
def powers_of_2(n):
    power = 1
    for i in range(n):
        yield power
        power *= 2


for v in powers_of_2(8):
    print(v)

1
2
4
8
16
32
64
128


Can you guess the output? Copy the code to the editor and run it to check your guesses.

### List comprehensions`

Generators may also be used within `list comprehensions`, just like here:

In [4]:
def powers_of_2(n):
    power = 1
    for i in range(n):
        yield power
        power *= 2


t = [x for x in powers_of_2(5)]
print(t)

[1, 2, 4, 8, 16]


Run the example and check the output.

### The list() function

The `list()` function can transform a series of subsequent generator invocations into `a real list`:

In [5]:
def powers_of_2(n):
    power = 1
    for i in range(n):
        yield power
        power *= 2


t = list(powers_of_2(3))
print(t)

[1, 2, 4]


Again, try to predict the output and run the code to check your predictions.

### The in operator

Moreover, the context created by the `in` operator allows you to use a generator, too.

The example shows how to do it:

In [6]:
def powers_of_2(n):
    power = 1
    for i in range(n):
        yield power
        power *= 2


for i in range(20):
    if i in powers_of_2(4):
        print(i)

1
2
4
8


What's the code's output? Run the program and check.

### The Fibanacci number generator

Now let's see a `Fibonacci number generator`, and ensure that it looks much better than the objective version based on the direct iterator protocol implementation.

Here it is:

In [7]:
def fibonacci(n):
    p = pp = 1
    for i in range(n):
        if i in [0, 1]:
            yield 1
        else:
            n = p + pp
            pp, p = p, n
            yield n

fibs = list(fibonacci(10))
print(fibs)


[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


Guess the output (a list) produced by the generator, and run the code to check if you were right.

# More about list comprehensions
You should be able to remember the rules governing the creation and use of a very special Python phenomenon named `list comprehension - a simple and very impressive way of creating lists and their contents`.

In case you need it, we've provided a quick reminder in the editor.
```py
list_1 = []

for ex in range(6):
    list_1.append(10 ** ex)

list_2 = [10 ** ex for ex in range(6)]

print(list_1)
print(list_2)
```
There are two parts inside the code, both creating a list containing a few of the first natural powers of ten.

The former uses a routine way of utilizing the `for` loop, while the latter makes use of the list comprehension and builds the list in situ, without needing a loop, or any other extended code.

It looks like the list is created inside itself - it's not true, of course, as Python has to perform nearly the same operations as in the first snippet, but it is indisputable that the second formalism is simply more elegant, and lets the reader avoid any unnecessary details.

The example outputs two identical lines containing the following text:
```s
[1, 10, 100, 1000, 10000, 100000]
[1, 10, 100, 1000, 10000, 100000]
```
Run the code to check if we're right.